In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
import pandas as pd

In [3]:
# Download the data - 106 GB !!!
# !kaggle competitions download -c siim-isic-melanoma-classification

In [4]:
path = Config.data_path()/'melclass'
path.mkdir(parents=True, exist_ok=True)
path
bs = 64
imsize = 114
valid_pct = .5

In [7]:
data = ImageDataBunch.from_csv(
    path, 
    'jpeg/train_resized', 
    csv_labels='train.csv',
    suffix='.jpg', 
    valid_pct=valid_pct,
    fn_col=0,
    label_col=7,
    size=imsize, # 224
    bs=bs,
    ds_tfms=get_transforms()
).normalize(imagenet_stats) # need to use imagenet stats because this is the one the model was trained with

In [31]:
# data.show_batch(rows=5, figsize=(7,6))

In [8]:
print(data.classes)
len(data.classes),data.c

[0, 1]


(2, 2)

In [9]:
learn = cnn_learner(data, models.resnet50, metrics=[error_rate, AUROC()])
# Could try resnet101 or 152. Also densenet etc.

In [36]:
# interp = ClassificationInterpretation.from_learner(learn)
# losses,idxs = interp.top_losses()
# len(data.valid_ds)==len(losses)==len(idxs)

In [37]:
# interp.plot_top_losses(9, figsize=(15,11))

In [38]:
# interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [39]:
# learn.lr_find()

In [40]:
# learn.recorder.plot()

In [41]:
# learn.unfreeze()
# s = slice(1e-3/(2.6**4),1e-3)
# learn.fit_one_cycle(10, max_lr=s)
# # learn.save('melclass-stage-2-resnet50');

Note, can also use freeze_to to unfreeze last x layers (e.g. freeze_to(-2) to unfreeze the last two layers)) - perhaps only relevant for NLP

OK now continue to train overnight using the entire training data.

In [14]:
data = ImageDataBunch.from_csv(
    path, 
    'jpeg/train_resized', 
    csv_labels='train.csv', # use full training data
    suffix='.jpg', 
    valid_pct=valid_pct,
    fn_col=0,
    label_col=7,
    size=imsize, # 224
    bs=bs,
    ds_tfms=get_transforms()
).normalize(imagenet_stats) # need to use imagenet stats because this is the one the model was trained with

learn = cnn_learner(data, models.resnet18, metrics=[error_rate, AUROC()])

In [15]:
s = slice(1e-3/(2.6**4),1e-3)
learn.fit_one_cycle(1, s)

epoch,train_loss,valid_loss,error_rate,auroc,time
0,0.249820,0.114529,0.018354,0.621555,01:39


In [16]:
learn.export(file = 'resnet18.pkl')

In [17]:

imlist = ImageList.from_csv(
    path/'jpeg/train',
    csv_name = path/'subset.csv',
    suffix = '.jpg'
)

learner = load_learner(path, file = 'resnet18.pkl', test=imlist)

learner.predict(imlist[0])

(Category tensor(0), tensor(0), tensor([0.9625, 0.0375]))

In [ ]:
learn.unfreeze()

learn.fit_one_cycle(10, max_lr=s, wd=1e-1)

learn.save('melclass-stage-2-resnet18-full');
learn.export(file = 'resnet18-stage2.pkl')

epoch,train_loss,valid_loss,error_rate,auroc,time
0,0.147634,0.087933,0.017449,0.666623,00:43
1,0.117265,0.084669,0.018475,0.800659,00:43
2,0.081313,0.075432,0.017509,0.825944,00:43
3,0.077664,0.076783,0.017630,0.854109,00:43


In [58]:
learner = load_learner(path, file = 'resnet18-stage2.pkl', test=imlist)
preds, y = learner.get_preds(ds_type = DatasetType.Test)

In [60]:
imagenames = [str(l).split('/')[-1].replace('.jpg','') for l in imlist.items]
predcol = [p for p in preds.numpy()[:,1]]
df_preds = pd.DataFrame({'image_name':imagenames, 'prob':predcol, 'modclass': y.numpy()})
df_preds.head()

,image_name,prob,modclass
0,ISIC_6468427,0.000587,0
1,ISIC_1217717,0.039139,0
2,ISIC_0355023,0.038888,0
3,ISIC_7508527,0.016699,0
4,ISIC_2844085,0.059611,0


In [61]:
df = pd.read_csv(path/'train.csv')
dfout = df.merge(df_preds, on='image_name')
dfout.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,prob,modclass
0,ISIC_0077472,IP_3691360,female,40.0,torso,unknown,benign,0,0.001104,0
1,ISIC_0080512,IP_1870306,male,75.0,torso,unknown,benign,0,0.001558,0
2,ISIC_0096328,IP_9435698,female,75.0,torso,unknown,benign,0,0.003596,0
3,ISIC_0109869,IP_8171635,male,70.0,torso,unknown,benign,0,0.000460,0
4,ISIC_0149568,IP_0962375,female,55.0,upper extremity,melanoma,malignant,1,0.001788,0


In [62]:
dfout.to_csv(path/'predictions-resnet18-subset.csv', index=False)

# Check for multiple images from each patient

In [67]:
path/'predictions-resnet18-subset.csv'

PosixPath('/home/jupyter/.fastai/data/melclass/predictions-resnet18-subset.csv')

In [66]:
# pd.set_option('display.max_rows', 20000)
# dfagg = df.groupby('patient_id').agg(
#     rowcount = ('patient_id', 'count'),
#     ntarget =  ('target', 'sum')
# )
# dfagg['ratio'] = dfagg.ntarget / dfagg.rowcount
# dfagg.sort_values('ratio', ascending=False)

,rowcount,ntarget,ratio
patient_id,,,
IP_2516168,3,3,1.000000
IP_5399626,5,4,0.800000
IP_3010556,4,3,0.750000
IP_4327880,4,3,0.750000
IP_7650895,3,2,0.666667
IP_7666205,3,2,0.666667
IP_0957064,3,2,0.666667
IP_5609476,3,2,0.666667
IP_4611463,3,2,0.666667
